In [5]:
from datetime import datetime
from os.path import join
from os import listdir
import re #for camel case conversion
import json
import ast

import numpy as np
import pandas as pd

import torch
from sklearn.preprocessing import LabelEncoder
import os
from os.path import join
from datetime import datetime
from os import listdir
import re #for camel case conversion
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

os.environ['LDA_name'] = 'num-directstr_thr-0_tn-400'

from extract.feature_extraction.topic_features_LDA import extract_topic_features
from extract.feature_extraction.sherlock_features import extract_sherlock_features
from utils import get_valid_types
from model import models_sherlock
from model.torchcrf import CRF

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
def camel_case(s):
  s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

def sherlock_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    s = ''.join([s[0].lower(), s[1:]])
    s = ''.join(map(lambda x: x if x.islower() else " "+x, s))
    return s

In [3]:
TYPENAME = os.environ['TYPENAME']
valid_types = get_valid_types(TYPENAME)
print(valid_types)
label_enc = LabelEncoder()
label_enc.fit(valid_types)

MAX_COL_COUNT = 10
topic_dim = 400
pre_trained_loc = './pretrained_sato'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("PyTorch device={}".format(device))
device = torch.device('cpu')

['address', 'affiliate', 'affiliation', 'age', 'album', 'area', 'artist', 'birthDate', 'birthPlace', 'brand', 'capacity', 'category', 'city', 'class', 'classification', 'club', 'code', 'collection', 'command', 'company', 'component', 'continent', 'country', 'county', 'creator', 'credit', 'currency', 'day', 'depth', 'description', 'director', 'duration', 'education', 'elevation', 'family', 'fileSize', 'format', 'gender', 'genre', 'grades', 'isbn', 'industry', 'jockey', 'language', 'location', 'manufacturer', 'name', 'nationality', 'notes', 'operator', 'order', 'organisation', 'origin', 'owner', 'person', 'plays', 'position', 'product', 'publisher', 'range', 'rank', 'ranking', 'region', 'religion', 'requirement', 'result', 'sales', 'service', 'sex', 'species', 'state', 'status', 'symbol', 'team', 'teamName', 'type', 'weight', 'year']


In [4]:
feature_group_cols = {}
sherlock_feature_groups = ['char', 'word', 'par', 'rest']
for f_g in sherlock_feature_groups:
    feature_group_cols[f_g] = list(pd.read_csv(join(os.environ['BASEPATH'],
                                          'configs', 'feature_groups', 
                                          "{}_col.tsv".format(f_g)),
                                           sep='\t', header=None, 
                                           index_col=0)[1])

In [5]:
pad_vec = lambda x: np.pad(x, (0, topic_dim - len(x)),
                    'constant',
                    constant_values=(0.0, 1/topic_dim))

In [6]:
classifier = models_sherlock.build_sherlock(sherlock_feature_groups, num_classes=len(valid_types), topic_dim=topic_dim, dropout_ratio=0.35)
#classifier.load_state_dict(torch.load(join(pre_trained_loc, 'sherlock_None.pt'), map_location=device))
model = CRF(len(valid_types) , batch_first=True).to(device)
#model.load_state_dict(torch.load(join(pre_trained_loc, 'model.pt'), map_location=device))

loaded_params = torch.load(join(pre_trained_loc, 'model.pt'), map_location=device)
classifier.load_state_dict(loaded_params['col_classifier'])
model.load_state_dict(loaded_params['CRF_model'])

classifier.eval()
model.eval()

CRF(num_tags=78)

In [7]:
def extract(df):

    df_dic = {'df':df, 'locator':'None', 'dataset_id':'None'}
    feature_dic = {}
    n = df.shape[1]

    # topic vectors
    topic_features = extract_topic_features(df_dic)
    topic_vec = pad_vec(topic_features.loc[0,'table_topic'])
    feature_dic['topic'] = torch.FloatTensor(np.vstack((np.tile(topic_vec,(n,1)), np.zeros((MAX_COL_COUNT - n, topic_dim)))))


    # sherlock vectors
    sherlock_features = extract_sherlock_features(df_dic)
    for f_g in feature_group_cols:
        temp = sherlock_features[feature_group_cols[f_g]].to_numpy()
        temp = np.vstack((temp, np.zeros((MAX_COL_COUNT - n, temp.shape[1])))).astype('float')
        temp = np.nan_to_num(temp)
        feature_dic[f_g] = torch.FloatTensor(temp)

    # dictionary of features, labels, masks
    return feature_dic, np.zeros(MAX_COL_COUNT), torch.tensor([1]*n + [0]*(MAX_COL_COUNT-n), dtype=torch.uint8)

In [8]:
def evaluate(df):

    feature_dic, labels, mask = extract(df)

    emissions = classifier(feature_dic).view(1, MAX_COL_COUNT, -1)
    mask = mask.view(1, MAX_COL_COUNT)
    pred = model.decode(emissions, mask)[0]

    return label_enc.inverse_transform(pred)

In [6]:
path_out_true_types = '../../combined/results/true_types'
path_out_predictions = '../../combined/results/predictions'
path_out_temp = '../results/sato_sato'
path_data = '../table_data/sato_tables/all'
write_out_limit = 3000

In [10]:
predicted_labels = [] 
true_types = []
idx = 0
write_out_idx = 0
#loop over al subdirectories of path_data
for subdir in listdir(path_data):
    print(subdir)
    if(subdir == 'processed'): continue
    path_subdir = join(path_data, subdir)
    #loop over all files in subdir
    for file in listdir(path_subdir):
        if(idx % 100 == 0):
            print(idx)
        idx += 1
#         print(file)
        path_file = join(path_subdir, file)

        df = pd.read_csv(path_file)
        df_header = list(df)
#         if (len(df_header)>1):
#             print('table: '+ str(df_header) )
        prediction = evaluate(df)
        predicted_labels.extend(prediction)
        true_types.extend(df_header)
        if (idx % write_out_limit == 0):
#             print(predicted_labels)
#             print(len(predicted_labels))
#             print(true_types)
#             print(len(true_types))
            # write out predictions to parquet file
            print("writing out predictions...")
            df_pred = pd.DataFrame(predicted_labels, columns=['predicted_type'])
            # write out predictions, format: model_dataset_tableCount_writeOutCount.parquet
            df_pred.to_parquet(join(path_out_temp, 'sato_sato_'+ str(idx) + '_' +str(write_out_idx)+ '.parquet'))
            write_out_idx += 1
            predicted_labels = []
            true_types = []
            
df_pred = pd.DataFrame(predicted_labels, columns=['predicted_type'])
df_pred.to_parquet(join(path_out_temp, 'sato_sato_'+ str(idx) + '_' +str(write_out_idx)+ '.parquet'))

            
            
# for idx, col in enumerate(test_values['values']):
#     col = ast.literal_eval(col)
#     df = pd.DataFrame(columns=['col'], data=col)
#     prediction = evaluate(df)
#     prediction = str(prediction[0])
#     predicted_labels.append(prediction)
#     if(idx % 100 == 0):
#         print(idx)

processed
K1
0


/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/pandas/core/strings/object_array.py:90: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat, flags=flags)
/home/senn/virtualenvs/col2type/lib/python3.7/site-packages/pandas/core/strings/object_array.py:90: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat, flags=flags)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
writing out predictions...
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
writing out predictions...
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
writing out predictions...
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
writing out predictions...
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
writing out predictions...
15000
15100
15200
15300
15400
15500
15600
15700
K2
15800
15900
16000
16100
1620

In [10]:
files = listdir(path_out_temp)
files.sort(key=lambda x: int(x.split('_')[2]))
# print(files)
df_combined = pd.DataFrame()
for file in files:
    filepath = os.path.join(path_out_temp, file)
    df_partial = pd.read_parquet(filepath)
    df_combined = pd.concat([df_combined, df_partial], ignore_index=True, sort=False)

df_combined.columns = ['type']
# print(set(df_combined['type']))
# for val in df_combined['type']:
#     if '1' in val: print(val)
# df['type'].transform()
print(df_combined.tail())

{'publisher', 'nationality', 'rank', 'order', 'fileSize', 'classification', 'continent', 'birthDate', 'birthPlace', 'format', 'sales', 'weight', 'club', 'jockey', 'company', 'description', 'city', 'isbn', 'name', 'symbol', 'artist', 'notes', 'region', 'elevation', 'collection', 'area', 'component', 'service', 'category', 'organisation', 'requirement', 'country', 'genre', 'result', 'religion', 'family', 'command', 'duration', 'brand', 'team', 'code', 'capacity', 'status', 'ranking', 'affiliate', 'currency', 'affiliation', 'creator', 'education', 'position', 'owner', 'gender', 'type', 'album', 'address', 'year', 'day', 'person', 'language', 'teamName', 'credit', 'grades', 'depth', 'state', 'county', 'industry', 'class', 'operator', 'product', 'sex', 'director', 'origin', 'age', 'plays', 'range', 'location', 'manufacturer', 'species'}
            type
120604     state
120605       age
120606  location
120607  position
120608      name


In [11]:
# print(predicted_labels)
# print(true_types)

In [29]:
df_combined.to_parquet(join(path_out_predictions, "sato_sato.parquet"))